In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, LSTM, GRU, Embedding, Bidirectional, Input
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import to_categorical

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


#RNN, LSTM, GRU

SimpleRNN

In [ ]:
class custom_rnn():
  def __init__(self, input_dim, hidden_dim, activation=np.tanh):
    self.h = np.zeros(hidden_dim)
    self.W = np.random.normal(size=(input_dim+hidden_dim, hidden_dim))
    self.b = np.random.normal(size=hidden_dim)
    self.activation = activation

  def __call__(self, x):
    self.h = self.activation(np.concatenate([self.h, x]) @ self.W + self.b)
    return self.h

In [ ]:
rnn = custom_rnn(5, 10)
x = np.array([1.,2,3,4,5])

In [ ]:
y = rnn(x)
y

array([ 0.99999312,  0.99733949,  0.87679561,  0.99973466,  0.99997538,
       -0.97106026, -0.99999998, -0.99999921,  0.98786393, -1.        ])

In [ ]:
rnn.h

array([ 0.99999312,  0.99733949,  0.87679561,  0.99973466,  0.99997538,
       -0.97106026, -0.99999998, -0.99999921,  0.98786393, -1.        ])

LSTM

In [ ]:
def custom_sigmoid(x):
  return 1/(1-np.exp(-x))

In [ ]:
custom_sigmoid(-np.inf), custom_sigmoid(np.inf)

(-0.0, 1.0)

In [ ]:
class custom_lstm():
  def __init__(self, input_dim, hidden_dim):
    self.h = np.zeros(hidden_dim)
    self.c = np.zeros(hidden_dim)

    self.Wf = np.random.normal(size=(input_dim+hidden_dim, hidden_dim))
    self.Wi = np.random.normal(size=(input_dim+hidden_dim, hidden_dim))
    self.Wo = np.random.normal(size=(input_dim+hidden_dim, hidden_dim))
    self.Wc = np.random.normal(size=(input_dim+hidden_dim, hidden_dim))

    self.bf = np.random.normal(size=hidden_dim)
    self.bi = np.random.normal(size=hidden_dim)
    self.bo = np.random.normal(size=hidden_dim)
    self.bc = np.random.normal(size=hidden_dim)


    self.tanh = np.tanh
    self.sigmoid = custom_sigmoid

  def __call__(self, x):
    hx = np.concatenate([self.h, x])

    f = self.sigmoid(hx @ self.Wf + self.bf)
    i = self.sigmoid(hx @ self.Wi + self.bi)
    o = self.sigmoid(hx @ self.Wo + self.bo)
    c = self.tanh(hx @ self.Wc + self.bc)

    self.c = f*self.c
    self.c += i*c
    
    self.h = o*self.tanh(self.c)

    return self.h

In [ ]:
lstm = custom_lstm(5, 10)
x = np.array([1.,2,3,4,5])

In [ ]:
y = lstm(x)
y

array([-4.42237987e-06, -7.21581977e-05,  3.93592704e-04, -8.02153050e-03,
        3.43115778e-02, -1.14421989e-02,  2.72993866e-02,  2.23411770e-03,
       -1.27461831e-03,  8.14326685e-04])

GRU

In [ ]:
class custom_gru():
  def __init__(self, input_dim, hidden_dim):
    self.h = np.zeros(hidden_dim)

    self.Wz = np.random.normal(size=(input_dim+hidden_dim, hidden_dim))
    self.Wr = np.random.normal(size=(input_dim+hidden_dim, hidden_dim))
    self.Wh = np.random.normal(size=(input_dim+hidden_dim, hidden_dim))


    self.bz = np.random.normal(size=hidden_dim)
    self.br = np.random.normal(size=hidden_dim)
    self.bh = np.random.normal(size=hidden_dim)


    self.tanh = np.tanh
    self.sigmoid = custom_sigmoid

  def __call__(self, x):
    hx = np.concatenate([self.h, x])

    
    r = self.sigmoid(hx @ self.Wr + self.br)
    hrx = np.concatenate([r*self.h, x])

    h = self.tanh(hrx @ self.Wh + self.bh)

    alpha = self.sigmoid(hx @ self.Wz + self.bz)

    self.h = (1-alpha)*self.h + alpha*h

    return self.h

In [ ]:
gru = custom_gru(5, 10)
x = np.array([1.,2,3,4,5])

In [ ]:
y = gru(x)
y

array([ 5.21105252e-04, -1.03642916e+00, -8.81214775e-01, -4.48960456e-05,
        9.97953757e-01,  1.20364390e-04,  1.74507097e+00,  1.81285700e+00,
        1.00175174e+00,  1.04907587e+00])

#Режимы работы RNN

In [ ]:
num_words = 100

In [ ]:
x = np.random.randint(0, 100, [3, 10]) #3 предложения по 10 токенов каждый

In [ ]:
emb = Embedding(100, 5) #для словаря из 100 слов. длина вектора - 5

In [ ]:
x_ = emb(x)
x_.shape #3 предложения по 10 слов, на каждое слово эмбеддинг длиной 5

TensorShape([3, 10, 5])

стандартный (выход после обработки всей последовательности)

In [ ]:
lstm = LSTM(10)
y = lstm(x_)
y.shape #3 предложения, финальное состояние длиной 10

TensorShape([3, 10])

return_sequences = True

In [ ]:
lstm = LSTM(10, return_sequences=True)
y = lstm(x_)
y.shape #3 предложения по 10 слов, входу каждого слова соответствует выход слоя размером 10

TensorShape([3, 10, 10])

return_state = True

In [ ]:
lstm = LSTM(10, return_state=True)
y, h, c = lstm(x_)
y.shape, h.shape, c.shape #список из трех состояний y, h, c, (y==h), для 3 предложений с финальным состоянием длиной 10

(TensorShape([3, 10]), TensorShape([3, 10]), TensorShape([3, 10]))

In [ ]:
(y == h).numpy().all()

True

In [ ]:
gru = GRU(10, return_state=True)
y, h = gru(x_)
y.shape, h.shape #список из двух состояний y, h, (y==h). для 3 предложений с финальным состоянием длиной 10

(TensorShape([3, 10]), TensorShape([3, 10]))

In [ ]:
(y == h).numpy().all()

True

go_backwards = True

In [ ]:
lstm = LSTM(10, go_backwards=True) #перед обработкой переворачивать последовательность
y = lstm(x_)
y.shape #3 предложения, финальное состояние каждого длиной 10

TensorShape([3, 10])

## Stateful RNN

stateful = False

In [ ]:
lstm = LSTM(10)
a = lstm(x_) #после выполнения состояния обнулились
b = lstm(x_) #после выполнения состояния обнулились
(a==b).numpy().all()

True

stateful = True

In [ ]:
lstm = LSTM(10, stateful=True) #изначально состояния нулевые (если не указан другой initializer)
a = lstm(x_) #после выполнения изначельное состояния сохранились
b = lstm(x_) #этот слой возьмет состояния, изменившиеся после предыдущего прохода
(a==b).numpy().all()

False

##bidirectional = True. (lstm1 & lstm2(go_backwards))

In [ ]:
lstm =  Bidirectional(LSTM(10))
y = lstm(x_)
y.shape #3 предложения, финальное состояние каждого длиной 20 (10 от прямой последовательности, 10 от перевернутой)

TensorShape([3, 20])

In [ ]:
lstm =  Bidirectional(LSTM(10, return_sequences=True))
y = lstm(x_)
y.shape #3 предложения по 10 слов состояние каждого длиной 20 (10 от прямой последовательности, 10 от перевернутой)

TensorShape([3, 10, 20])